In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import factors
from sklearn.linear_model import LinearRegression

In [5]:
def getmacros():
    '''
    Returns a dataframe of excess returns from macro factors
    '''
    xls_file = r'C:\Users\bmcdonald\Desktop\MSCI Factors.xlsm'
    xls = pd.ExcelFile(xls_file)
    macros = xls.parse('AssetClass', index_col=0, header=None, skiprows=1,
                       names='MKT INT CRD HY CUR RF'.split())
    macros['CRD'] = 0.5*macros.CRD + 0.5*macros.HY
    macros.drop('HY', axis=1, inplace=True)
    macros['RF'] = macros['RF'] / 12  #3m treasuries are annualized, so this converts to monthly
    macros.index = pd.to_datetime(macros.index).strftime('%Y-%m')
    macros = factors.isolate_credit(macros)
    macros.dropna(inplace=True)
    return macros

In [2]:
def macro_prep(return_series, macros, start=None, end=None):
    
    returns = pd.DataFrame(return_series)

    # Opens macro factors, concats return series, subs rf, drops rf

    df = pd.concat([macros, returns], axis=1, sort=False)
    df.iloc[:,-1] = df.iloc[:,-1] - df.RF
    
    # Adjust Date range here if applicable
    df = df.loc[start:end]
    df.dropna(inplace=True)
    
    ex_macros = df.drop('RF', axis=1)



    # Creates separate df for excess returns
    ex_returns = ex_macros.iloc[:, -1]
    ex_returns = pd.DataFrame(ex_returns)
    assert ex_returns.shape[0] > 0, 'Manager returns df is empty'

    ex_macros = ex_macros.iloc[:, :4]
    assert ex_macros.shape[0] > 0, 'Macro returns df is empty'

    return ex_returns, ex_macros
    
    

In [3]:
xls_file = r'C:\Users\bmcdonald\Desktop\OMG Manager Returns.xlsx'
xls = pd.ExcelFile(xls_file)


In [8]:
import datetime
today = datetime.date.today()
first = today.replace(day=1)
lastMonth = first - datetime.timedelta(days=1)
date = lastMonth.strftime("%Y-%m")

beta_df = pd.DataFrame(columns=['MKT INT CRD CUR'.split()])
macros = getmacros()

excess_returns_df = pd.DataFrame()
predict_df = pd.DataFrame()
error_df = pd.DataFrame()

ols = LinearRegression()

for category in 'Index Fund Alt'.split():
    
    cat = xls.parse(category, index_col=0, skiprows=1)
    cat.index = pd.to_datetime(cat.index).strftime('%Y-%m')
    name = cat.columns
    
    start = cat.index[120]
    
    betas = []
    
    for n in name:
        # Uses factorprep to calc excess manager and factor returns
        ex_returns, ex_macros = macro_prep(cat[n], macros, start)
        
        model = ols.fit(ex_macros.values,
                        ex_returns.values)
        
        beta = model.coef_.flatten()
        betas.append(beta)
        
        errors = ex_returns - model.predict(ex_macros)
        predictions = ex_returns - errors 
        
        excess_returns_df = pd.concat([excess_returns_df, ex_returns], axis=1, sort=False)
        predict_df = pd.concat([predict_df, predictions], axis=1, sort=False)
        error_df = pd.concat([error_df, errors], axis=1, sort=False)
        

    df = pd.DataFrame(np.stack(betas), columns=['MKT INT CRD CUR'.split()], index=name)

    beta_df = pd.concat([beta_df, df], sort=False)
    
beta_df.to_excel(f'{date} - TEST Macro Betas.xlsx')


excess_returns_df.sort_index(inplace=True)
predict_df.sort_index(inplace=True)
error_df.sort_index(inplace=True)

excess_returns_df.to_excel(f'{date} - TEST Excess Returns.xlsx')
predict_df.to_excel(f'{date} - TEST Model Predictions.xlsx')
error_df.to_excel(f'{date} - TEST Errors.xlsx')
#error_df.corr().to_excel(f'{date} - TEST Macro Error Correlation.xlsx')

## This section is going to estimate past betas to analyze what return predicted returns would be to compare with actual returns

In [51]:
xls_file = r'C:\Users\bmcdonald\Desktop\OMG Manager Returns.xlsx'
xls = pd.ExcelFile(xls_file)


returns = xls.parse('Index', index_col=0, skiprows=1)
returns.index = pd.to_datetime(returns.index).strftime('%Y-%m')

funds = xls.parse('Fund', index_col=0, skiprows=1)
funds.index = pd.to_datetime(funds.index).strftime('%Y-%m')

returns = pd.concat([returns, funds], axis=1)
returns = returns.iloc[:,:-4]

dates = returns.index
names = returns.columns

macros = getmacros()

date_i = np.flip(np.arange(1,61))

In [42]:
pred_returns_df = pd.DataFrame(columns=names)

for end_date_i in date_i:
    
    start_date_i = end_date_i + 120
    returns_10y = returns.iloc[end_date_i:start_date_i] #Dates are in descending order
    
    macros_tplus1 = macros.loc[dates[end_date_i-1]]
    
    pred_returns=[]
    
    
    for name in names:
        
        ex_returns, ex_macros = macro_prep(returns[name], macros)
        
        model = ols.fit(ex_macros.values,
                        ex_returns.values)
        
        beta = model.coef_.flatten()
        
        pred_returns.append(beta[0] * macros_tplus1[0] +
                            beta[1] * macros_tplus1[1] +
                            beta[2] * macros_tplus1[2] +
                            beta[3] * macros_tplus1[3])
    
    pred_returns_df.loc[dates[end_date_i - 1]] = pred_returns

In [52]:
returns = returns.loc[:'2015-11'][::-1]

forecasting_errors = returns - pred_returns_df

In [55]:
pred_returns_df.to_excel("2020-10 - Forward Looking Return Estimates.xlsx")
forecasting_errors.to_excel("2020-10 - Forward Looking Return Errors.xlsx")